In [1]:
import  matplotlib.pyplot as plt  
import  numpy as npi
import math
from gurobipy import *

# Lectura Archivo FJS

In [2]:
direcionFJS="/Users\PC\OneDrive - Universidad de la Sabana\MDGP\Combined Job-Shop and VRP\Instances example\Sfjs03.txt"
#"/Users\PC\OneDrive - Universidad de la Sabana\MDGP\Conference paper\solomon\edata\ela01.fjs"
instanciaFJS = open(direcionFJS)
contador=0
contadorPedidos=0
pedidos=[]
maquinas=[]
operacionesxpedido=[]
TPA={}
numeroPedidos=0
numeroMaquinas=0
for  line in instanciaFJS:
    linea= line.split()
    if(contador==0):
        numeroPedidos=int(linea[0])
        numeroMaquinas=int(linea[1])
        maquinas=range(numeroMaquinas)
        pedidos=range(1,numeroPedidos+1)
        dicInitial= {}
        for m in range(numeroMaquinas):
            dicInitial[m]=0
        TPA[(0,0)]=dicInitial
    else:
        print(linea)
        numeroOperacionesPedido= int(linea[0])
        operacionesxpedido.append(numeroOperacionesPedido)
        contadorElementoLista=1
        for operacion in range(numeroOperacionesPedido):
            Maquinas= int(linea[contadorElementoLista])
            MachinesTime={}
            for maquina in range(Maquinas):
                maquina= int(linea[contadorElementoLista+1])-1
                tiempo= int(linea[contadorElementoLista+2])
                contadorElementoLista=contadorElementoLista+2
                MachinesTime[maquina]=tiempo
            contadorElementoLista=contadorElementoLista+1
            TPA[(contadorPedidos+1,operacion+1)]= MachinesTime
        contadorPedidos= contadorPedidos+1
    contador=contador+1
print(pedidos)

['3', '2', '1', '119', '2', '103', '1', '1', '111', '3', '3', '75', '1', '70', '2', '120']
['3', '1', '1', '93', '2', '1', '90', '2', '76', '1', '1', '89']
['3', '2', '1', '109', '2', '112', '1', '1', '113', '3', '2', '93', '3', '71', '1', '98']
['3', '2', '2', '79', '1', '107', '1', '1', '71', '1', '1', '103']
range(1, 5)


In [3]:
PM={}
TP={}
for i,j in TPA:
    arreglo={}
    ar2={}
    for machine in maquinas:
        bol= False
        for maquina in TPA[i,j]:
            if(machine==maquina):
                bol= True
        if(bol==True):
            arreglo[machine]=1
            ar2[machine]=TPA[i,j][machine]
        else:
            arreglo[machine]=0
            ar2[machine]=0
    PM[(i,j)]=arreglo
    TP[(i,j)]=ar2
print(TPA)
arcos=[(m,i,j,k,l) for m in maquinas for i,j in TPA for k,l in TPA ]

{(0, 0): {0: 0, 1: 0, 2: 0}, (1, 1): {0: 119, 1: 103}, (1, 2): {0: 111}, (1, 3): {2: 75, 0: 70, 1: 120}, (2, 1): {0: 93}, (2, 2): {0: 90, 1: 76}, (2, 3): {0: 89}, (3, 1): {0: 109, 1: 112}, (3, 2): {0: 113}, (3, 3): {1: 93, 2: 71, 0: 98}, (4, 1): {1: 79, 0: 107}, (4, 2): {0: 71}, (4, 3): {0: 103}}


# Creación del modelo

In [4]:
model = Model('FJSP-CVRPRT')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-02


In [5]:
x= model.addVars(arcos, vtype= GRB.BINARY, name='x')
sk= model.addVars([k for k in maquinas],vtype=GRB.INTEGER, name='sk')
P=model.addVars([(i,j) for i,j in TPA],vtype= GRB.INTEGER, name='P')
SM=model.addVars([(i,j,k) for i,j in TPA for k in maquinas],vtype= GRB.BINARY, name='Sm')
IT=model.addVars([(i,j) for i,j in TPA],vtype= GRB.INTEGER, name='IT')
TM=model.addVars([(m,i,j) for m in maquinas for i,j in TPA], name= 'TM')
ym = model.addVar(vtype=GRB.INTEGER, name= 'makespan')

# Agregando restriciones FJS

In [6]:
print(TPA)

{(0, 0): {0: 0, 1: 0, 2: 0}, (1, 1): {0: 119, 1: 103}, (1, 2): {0: 111}, (1, 3): {2: 75, 0: 70, 1: 120}, (2, 1): {0: 93}, (2, 2): {0: 90, 1: 76}, (2, 3): {0: 89}, (3, 1): {0: 109, 1: 112}, (3, 2): {0: 113}, (3, 3): {1: 93, 2: 71, 0: 98}, (4, 1): {1: 79, 0: 107}, (4, 2): {0: 71}, (4, 3): {0: 103}}


In [7]:
model.addConstrs(quicksum(SM[i,j,m] for m in maquinas)==1 for i,j in TPA if i>0)
model.addConstrs(PM[i,j][m]>= SM[i,j,m] for i,j in TPA for m in maquinas if i>0)

model.addConstrs((SM[i,j,k]==1) >> (P[i,j]==TP[i,j][k]) for i, j in TP for k in maquinas)
#model.addConstrs((P[i,j]>=TP[i,j][k]*SM[i,j,k]) for i, j in TP for k in maquinas)

model.addConstrs(IT[i,j]+P[i,j]<=IT[i,j+1] for i,j in TPA if(i>0) and (j<operacionesxpedido[i-1]))
model.addConstrs(SM[i,j,m]>=x[m,i,j,k,l]for m,i,j,k,l in arcos)


model.addConstrs(quicksum(x[m,i,j,k,l] for k,l in TPA if(k,l)!=(i,j))==SM[i,j,m] for m in maquinas for i,j in TPA if(i>0))
model.addConstrs(quicksum(x[m,i,j,k,l] for i,j in TPA if(k,l)!=(i,j))==SM[k,l,m] for m in maquinas for k,l in TPA if(k>0))

model.addConstrs(quicksum(x[m,0,0,k,l] for k,l in TPA)==1 for m in maquinas)
model.addConstrs(quicksum(x[m,i,j,0,0] for i,j in TPA)==1 for m in maquinas)

model.addConstrs((x[m,i,j,k,l]==1) >>(TM[m,k,l]>=TM[m,i,j] +P[i,j]) for m in maquinas for i,j in TPA for k,l in TPA if k>0)
model.addConstrs((x[m,i,j,k,l]==1) >>(TM[m,i,j]>=IT[i,j]) for m in maquinas for i,j in TPA for k,l in TPA)
model.addConstrs((x[m,i,j,k,l]==1) >>(IT[i,j]>=TM[m,i,j]) for m in maquinas for i,j in TPA for k,l in TPA)

Ci=model.addVars([(i) for i in range(numeroPedidos)],vtype=GRB.INTEGER)
model.addConstrs(Ci[i]==IT[i+1,operacionesxpedido[i]]+P[i+1,operacionesxpedido[i]] for i in range(numeroPedidos))
model.addConstrs(ym>=IT[i+1,operacionesxpedido[i]]+P[i+1,operacionesxpedido[i]]for i in range(numeroPedidos))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

# Lectura de datos VRP

In [8]:
direccionVRP='/Users\PC\OneDrive - Universidad de la Sabana\MDGP\Conference paper\solomon\SolomonIntance\c101.txt'
instancia = open(direccionVRP)
contador =0
ubicaciones={}
contadorOrder=0
contadorProduct=0
product=1
for  line in instancia:
    if contador >8 and contador<=(numeroPedidos+1)*product+8:
        linea= line.split()
        if(contadorProduct==product):
            contadorProduct=0
            contadorOrder=contadorOrder+1
        ubicaciones[(int(contadorOrder),int(contadorProduct))]={'x':int(linea[1]),'y':int(linea[2]), 'w':int(linea[3])}
        contadorProduct=contadorProduct+1
    contador=contador+1
Distancia={}
rutas=15
arcosVRP=[(r,i,j,k,l) for r in range(rutas) for i,j in ubicaciones for k,l in ubicaciones]
for ubicacion in ubicaciones:
    for ubicacion2 in ubicaciones:
        if(ubicacion!=ubicacion2):
            x1=int(ubicaciones[ubicacion]['x'])
            x2=int(ubicaciones[ubicacion2]['x'])
            y1=int(ubicaciones[ubicacion]['y'])
            y2=int(ubicaciones[ubicacion2]['y'])
            Distancia[(ubicacion, ubicacion2)]= int(round(npi.hypot(x1-x2, y1-y2))*60/17)

print(ubicaciones)

{(0, 0): {'x': 40, 'y': 50, 'w': 0}, (1, 0): {'x': 45, 'y': 68, 'w': 10}, (2, 0): {'x': 45, 'y': 70, 'w': 30}, (3, 0): {'x': 42, 'y': 66, 'w': 10}, (4, 0): {'x': 42, 'y': 68, 'w': 10}}


In [9]:
print(instancia)

<_io.TextIOWrapper name='/Users\\PC\\OneDrive - Universidad de la Sabana\\MDGP\\Conference paper\\solomon\\SolomonIntance\\c101.txt' mode='r' encoding='cp1252'>


# Restriciones VRP

Creación de variables

In [10]:
ITV=model.addVars([(r) for r in range(rutas)], vtype=GRB.INTEGER)
Bina=model.addVars([(i,s,r) for i,s in ubicaciones for r in range(rutas)], vtype=GRB.BINARY, name='B')
U=model.addVars([(r,i,s) for r in range(rutas) for i,s in ubicaciones], vtype=GRB.INTEGER)
xv= model.addVars(arcosVRP, vtype= GRB.BINARY, name='xv')
TTV=model.addVars([(r) for r in range(rutas)], vtype=GRB.INTEGER)
CRMAX= model.addVar(vtype=GRB.INTEGER)

Restriciones

In [11]:
model.addConstrs((Bina[i,s,r]==1)>>(ITV[r]>=Ci[i-1]) for i,s in ubicaciones for r in range(rutas) if i>0)
model.addConstrs(quicksum(Bina[i,s,r] for r in range(rutas))==1 for i,s in ubicaciones if i>0)
model.addConstrs(quicksum(xv[r,i,s,k,t] for k,t in ubicaciones if(k,t)!=(i,s))==Bina[i,s,r] for r in range(rutas) for i,s in ubicaciones if i>0)
model.addConstrs(quicksum(xv[r,i,s,k,t] for i,s in ubicaciones if(i,s)!=(k,t))==Bina[k,t,r] for r in range(rutas) for k,t in ubicaciones if k>0)
model.addConstrs(quicksum(xv[r,0,0,k,t] for k,t in ubicaciones)==1 for r in range(rutas))
model.addConstrs(quicksum(xv[r,i,s,0,0] for i,s in ubicaciones)==1 for r in range(rutas))
model.addConstrs((U[r,i,s]+1<=10*(1-xv[r,i,s,k,t])+U[r,k,t]) for r,i,s,k,t in arcosVRP if(i,s)!=(k,t) and k>0)
model.addConstrs((TTV[r]==quicksum(Distancia[(i,s),(k,t)]*xv[r,i,s,k,t] for i,s in ubicaciones
for k,t in ubicaciones if(i,s)!=(k,t))) for r in range(rutas))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>}

In [12]:
model.addConstrs(ITV[r]==0 for r in range(rutas) for i in ubicaciones if r==0)
model.addConstrs(ITV[r]==ITV[r-1]+120 for r in range(rutas) for i in ubicaciones if r>0)
model.addConstrs((CRMAX>=TTV[r]+ ITV[r]-1000000*xv[r,0,0,0,0]) for r in range(rutas))
#model.addConstrs((CRMAX>=TTV[r]+ ITV[r]) for r in range(rutas))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>}

#  Definiendo función objetivo

In [13]:
model.setObjective(CRMAX,GRB.MINIMIZE)
model.Params.MIPGAP=0.35

Set parameter MIPGap to value 0.35


# Correr el modelo

In [14]:
model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1148 rows, 1175 columns and 4198 nonzeros
Model fingerprint: 0xc469e9cd
Model has 1581 general constraints
Variable types: 39 continuous, 1136 integer (996 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
  GenCon rhs range [7e+01, 1e+02]
  GenCon coe range [1e+00, 1e+00]
Presolve added 0 rows and 81 columns
Presolve removed 112 rows and 0 columns
Presolve time: 0.06s
Presolved: 1036 rows, 1256 columns, 3886 nonzeros
Presolved model has 652 SOS constraint(s)
Variable types: 621 continuous, 635 integer (517 binary)

Root relaxation: objective 0.000000e+00, 459 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Nod

In [15]:
print(Ci)

{0: <gurobi.Var C615 (value 480.0)>, 1: <gurobi.Var C616 (value 426.0)>, 2: <gurobi.Var C617 (value 297.0)>, 3: <gurobi.Var C618 (value 600.0)>}


In [16]:
print(ITV)

{0: <gurobi.Var C619 (value 0.0)>, 1: <gurobi.Var C620 (value 120.0)>, 2: <gurobi.Var C621 (value 240.0)>, 3: <gurobi.Var C622 (value 360.0)>, 4: <gurobi.Var C623 (value 480.0)>, 5: <gurobi.Var C624 (value 600.0)>, 6: <gurobi.Var C625 (value 720.0)>, 7: <gurobi.Var C626 (value 840.0)>, 8: <gurobi.Var C627 (value 960.0)>, 9: <gurobi.Var C628 (value 1080.0)>, 10: <gurobi.Var C629 (value 1200.0)>, 11: <gurobi.Var C630 (value 1320.0)>, 12: <gurobi.Var C631 (value 1440.0)>, 13: <gurobi.Var C632 (value 1560.0)>, 14: <gurobi.Var C633 (value 1680.0)>}


In [17]:
print(TTV)

{0: <gurobi.Var C1159 (value 0.0)>, 1: <gurobi.Var C1160 (value 0.0)>, 2: <gurobi.Var C1161 (value 0.0)>, 3: <gurobi.Var C1162 (value 0.0)>, 4: <gurobi.Var C1163 (value 148.0)>, 5: <gurobi.Var C1164 (value 126.0)>, 6: <gurobi.Var C1165 (value 0.0)>, 7: <gurobi.Var C1166 (value 0.0)>, 8: <gurobi.Var C1167 (value 0.0)>, 9: <gurobi.Var C1168 (value 0.0)>, 10: <gurobi.Var C1169 (value 0.0)>, 11: <gurobi.Var C1170 (value 0.0)>, 12: <gurobi.Var C1171 (value 0.0)>, 13: <gurobi.Var C1172 (value 0.0)>, 14: <gurobi.Var C1173 (value 0.0)>}


In [18]:
for arc in arcosVRP:
    if (xv[arc].x>0):
        print(xv[arc] )

<gurobi.Var xv[0,0,0,0,0] (value 1.0)>
<gurobi.Var xv[1,0,0,0,0] (value 1.0)>
<gurobi.Var xv[2,0,0,0,0] (value 1.0)>
<gurobi.Var xv[3,0,0,0,0] (value 1.0)>
<gurobi.Var xv[4,0,0,2,0] (value 1.0)>
<gurobi.Var xv[4,1,0,0,0] (value 1.0)>
<gurobi.Var xv[4,2,0,1,0] (value 1.0)>
<gurobi.Var xv[5,0,0,4,0] (value 1.0)>
<gurobi.Var xv[5,3,0,0,0] (value 1.0)>
<gurobi.Var xv[5,4,0,3,0] (value 1.0)>
<gurobi.Var xv[6,0,0,0,0] (value 1.0)>
<gurobi.Var xv[7,0,0,0,0] (value 1.0)>
<gurobi.Var xv[8,0,0,0,0] (value 1.0)>
<gurobi.Var xv[9,0,0,0,0] (value 1.0)>
<gurobi.Var xv[10,0,0,0,0] (value 1.0)>
<gurobi.Var xv[11,0,0,0,0] (value 1.0)>
<gurobi.Var xv[12,0,0,0,0] (value 1.0)>
<gurobi.Var xv[13,0,0,0,0] (value 1.0)>
<gurobi.Var xv[14,0,0,0,0] (value 1.0)>


In [19]:
for m in maquinas:
    Linea='Machine '+str(m)+':'
    initial = (0,0)
    follow=(0,0)
    Continuar=True
    while(Continuar):
        initial=follow
        for k,l in TPA:
            i=initial[0]
            j=initial[1]
            if (x[m,i,j,k,l].x>0.99):
                final=abs(IT[i,j].x+P[i,j].x)
                if(initial!=(0,0)):
                    Linea=Linea+ '('+str(i)+'-'+str(j)+')'+'('+str(abs(IT[i,j].x))+'-'+str(final)+') '
                follow=(k,l)
        if(follow==(0,0)):
            Continuar=False
    print(Linea)

Machine 0:(2-1)(0.0-93.0) (3-2)(112.99999999999997-225.99999999999997) (1-2)(225.99999999999997-337.0) (2-3)(337.0-426.0) (4-2)(426.0-497.0) (4-3)(497.0-600.0) 
Machine 1:(3-1)(0.9999999999999716-112.99999999999997) (1-1)(122.99999999999997-225.99999999999997) (2-2)(261.0-337.0) (4-1)(347.0-426.0) 
Machine 2:(3-3)(225.99999999999997-297.0) (1-3)(405.0-480.0) 


In [20]:
for r in range(rutas):
    Linea='Route '+str(r)+':'
    print(Linea)
    initial = (0,0)
    follow=(0,0)
    Continuar=True
    tiempoVisita= ITV[r].x
    while(Continuar):
        initial=follow
        for k,l in ubicaciones:
            i,j=initial
            if (xv[r,i,j,k,l].x>0.99):
                print('['+str(initial)+']'+'['+str(ubicaciones[i,j]['x'])+', '+str(ubicaciones[i,j]['y'])+', '+str(tiempoVisita)+']-'+str(U[r,i,j]))
                try:
                    tiempoVisita=tiempoVisita+Distancia[(i,j),(k,l)]
                except:
                    print(0)
                follow=(k,l)
        if(follow==(0,0)):
            print('['+str(follow)+']'+'['+str(ubicaciones[0,0]['x'])+', '+str(ubicaciones[0,0]['y'])+', '+str(tiempoVisita)+']-'+str(U[r,i,j]))
            Continuar=False
    

Route 0:
[(0, 0)][40, 50, 0.0]-<gurobi.Var C709 (value -0.0)>
0
[(0, 0)][40, 50, 0.0]-<gurobi.Var C709 (value -0.0)>
Route 1:
[(0, 0)][40, 50, 120.0]-<gurobi.Var C714 (value -0.0)>
0
[(0, 0)][40, 50, 120.0]-<gurobi.Var C714 (value -0.0)>
Route 2:
[(0, 0)][40, 50, 240.0]-<gurobi.Var C719 (value -0.0)>
0
[(0, 0)][40, 50, 240.0]-<gurobi.Var C719 (value -0.0)>
Route 3:
[(0, 0)][40, 50, 360.0]-<gurobi.Var C724 (value -0.0)>
0
[(0, 0)][40, 50, 360.0]-<gurobi.Var C724 (value -0.0)>
Route 4:
[(0, 0)][40, 50, 480.0]-<gurobi.Var C729 (value -0.0)>
[(2, 0)][45, 70, 554.0]-<gurobi.Var C731 (value 1.0)>
[(1, 0)][45, 68, 561.0]-<gurobi.Var C730 (value 2.0)>
[(0, 0)][40, 50, 628.0]-<gurobi.Var C730 (value 2.0)>
Route 5:
[(0, 0)][40, 50, 600.0]-<gurobi.Var C734 (value -0.0)>
[(4, 0)][42, 68, 663.0]-<gurobi.Var C738 (value 1.0)>
[(3, 0)][42, 66, 670.0]-<gurobi.Var C737 (value 2.0)>
[(0, 0)][40, 50, 726.0]-<gurobi.Var C737 (value 2.0)>
Route 6:
[(0, 0)][40, 50, 720.0]-<gurobi.Var C739 (value -0.0)>
0
[(

In [21]:
for i in ubicaciones:
    print(i)

(0, 0)
(1, 0)
(2, 0)
(3, 0)
(4, 0)


In [22]:
print(ym)

<gurobi.Var makespan (value 600.0)>


In [23]:
print(xv)

{(0, 0, 0, 0, 0): <gurobi.Var xv[0,0,0,0,0] (value 1.0)>, (0, 0, 0, 1, 0): <gurobi.Var xv[0,0,0,1,0] (value 0.0)>, (0, 0, 0, 2, 0): <gurobi.Var xv[0,0,0,2,0] (value 0.0)>, (0, 0, 0, 3, 0): <gurobi.Var xv[0,0,0,3,0] (value 0.0)>, (0, 0, 0, 4, 0): <gurobi.Var xv[0,0,0,4,0] (value 0.0)>, (0, 1, 0, 0, 0): <gurobi.Var xv[0,1,0,0,0] (value 0.0)>, (0, 1, 0, 1, 0): <gurobi.Var xv[0,1,0,1,0] (value 0.0)>, (0, 1, 0, 2, 0): <gurobi.Var xv[0,1,0,2,0] (value 0.0)>, (0, 1, 0, 3, 0): <gurobi.Var xv[0,1,0,3,0] (value 0.0)>, (0, 1, 0, 4, 0): <gurobi.Var xv[0,1,0,4,0] (value 0.0)>, (0, 2, 0, 0, 0): <gurobi.Var xv[0,2,0,0,0] (value 0.0)>, (0, 2, 0, 1, 0): <gurobi.Var xv[0,2,0,1,0] (value 0.0)>, (0, 2, 0, 2, 0): <gurobi.Var xv[0,2,0,2,0] (value 0.0)>, (0, 2, 0, 3, 0): <gurobi.Var xv[0,2,0,3,0] (value 0.0)>, (0, 2, 0, 4, 0): <gurobi.Var xv[0,2,0,4,0] (value 0.0)>, (0, 3, 0, 0, 0): <gurobi.Var xv[0,3,0,0,0] (value 0.0)>, (0, 3, 0, 1, 0): <gurobi.Var xv[0,3,0,1,0] (value 0.0)>, (0, 3, 0, 2, 0): <gurobi.Var x

In [24]:
arcosVRP

[(0, 0, 0, 0, 0),
 (0, 0, 0, 1, 0),
 (0, 0, 0, 2, 0),
 (0, 0, 0, 3, 0),
 (0, 0, 0, 4, 0),
 (0, 1, 0, 0, 0),
 (0, 1, 0, 1, 0),
 (0, 1, 0, 2, 0),
 (0, 1, 0, 3, 0),
 (0, 1, 0, 4, 0),
 (0, 2, 0, 0, 0),
 (0, 2, 0, 1, 0),
 (0, 2, 0, 2, 0),
 (0, 2, 0, 3, 0),
 (0, 2, 0, 4, 0),
 (0, 3, 0, 0, 0),
 (0, 3, 0, 1, 0),
 (0, 3, 0, 2, 0),
 (0, 3, 0, 3, 0),
 (0, 3, 0, 4, 0),
 (0, 4, 0, 0, 0),
 (0, 4, 0, 1, 0),
 (0, 4, 0, 2, 0),
 (0, 4, 0, 3, 0),
 (0, 4, 0, 4, 0),
 (1, 0, 0, 0, 0),
 (1, 0, 0, 1, 0),
 (1, 0, 0, 2, 0),
 (1, 0, 0, 3, 0),
 (1, 0, 0, 4, 0),
 (1, 1, 0, 0, 0),
 (1, 1, 0, 1, 0),
 (1, 1, 0, 2, 0),
 (1, 1, 0, 3, 0),
 (1, 1, 0, 4, 0),
 (1, 2, 0, 0, 0),
 (1, 2, 0, 1, 0),
 (1, 2, 0, 2, 0),
 (1, 2, 0, 3, 0),
 (1, 2, 0, 4, 0),
 (1, 3, 0, 0, 0),
 (1, 3, 0, 1, 0),
 (1, 3, 0, 2, 0),
 (1, 3, 0, 3, 0),
 (1, 3, 0, 4, 0),
 (1, 4, 0, 0, 0),
 (1, 4, 0, 1, 0),
 (1, 4, 0, 2, 0),
 (1, 4, 0, 3, 0),
 (1, 4, 0, 4, 0),
 (2, 0, 0, 0, 0),
 (2, 0, 0, 1, 0),
 (2, 0, 0, 2, 0),
 (2, 0, 0, 3, 0),
 (2, 0, 0, 4, 0),
 (2, 1, 0,

In [25]:
print(ym)

<gurobi.Var makespan (value 600.0)>
